# Машинное обучение ранжированию с помощью библиотеки CatBoost

В этом примере мы:
- увидим как выглядят датасеты для машинного обучения ранжированию, на примере стандартного датасета [MSLR](https://www.microsoft.com/en-us/research/project/mslr/)
- познакомимся с библиотекой **CatBoost**
- научимся решать задачу машинного обучения ранжирования используя алгоритмы, реализованные в **CatBoost**

## Библиотека CatBoost

Полезные ссылки:
- домашняя страница: https://catboost.ai/
- официальный тюториал по машинному обучению ранжированию: https://github.com/catboost/tutorials/blob/master/ranking/ranking_tutorial.ipynb
- полный список доступных для оптимизации метрик и лоссов: https://catboost.ai/en/docs/concepts/loss-functions-ranking

## Пререквизиты

Импортируем все что нам понадобится для дальнейшей работы:

In [1]:
import copy
from timeit import default_timer as timer

import catboost
from catboost import datasets, utils

## Датасет MSLR (Microsoft Learning to Rank)

Дальше мы будем работать с датасетом MSLR.

Полный датасет можно скачать с официального сайта: https://www.microsoft.com/en-us/research/project/mslr/

Мы этого делать не будем т.к. в CatBoost уже встроена возможность загрузить небольшой сабсет MSLR, с которым мы и будем работать дальше.

Загрузим этот сабсет:

In [2]:
df_train, df_test = datasets.msrank_10k()

Датасет представляет собой обычный табличный датасет из 130 колонок:
- В первой колонке лежит таргет (оценка асессора), по 5-балльной шкале релевантности: от 0 до 4 (включительно)
- Во второй колонке лежит ID запроса, по которому можно сгруппировать все оценки документов в рамках одного и того же запроса
- Дальше идет вектор из 128 фичей (таких как значения BM25 и т.п.), их точная природа нам сейчас на важна

Посмотрим на данные:

In [3]:
print(df_train.head(5))

   0    1    2    3    4    5    6    7    8         9    ...  128       129  \
0  2.0    1    3    3    0    0    3  1.0  1.0  0.000000  ...   62  11089534   
1  2.0    1    3    0    3    0    3  1.0  0.0  1.000000  ...   54  11089534   
2  0.0    1    3    0    2    0    3  1.0  0.0  0.666667  ...   45         3   
3  2.0    1    3    0    3    0    3  1.0  0.0  1.000000  ...   56  11089534   
4  1.0    1    3    0    3    0    3  1.0  0.0  1.000000  ...   64         5   

   130  131    132  133  134  135  136  137  
0    2  116  64034   13    3    0    0  0.0  
1    2  124  64034    1    2    0    0  0.0  
2    1  124   3344   14   67    0    0  0.0  
3   13  123  63933    1    3    0    0  0.0  
4    7  256  49697    1   13    0    0  0.0  

[5 rows x 138 columns]


Для удобства присвоим колонкам говорящие имена:

In [4]:
def generate_column_names(num_features):
    """Generates column names for LETOR-like datasets"""
    columns = ['label', 'qid']
    for i in range(num_features):
        column = f"feature_{i+1}"
        columns.append(column)
    return columns

# Assign column names
columns = generate_column_names(num_features=136)
df_train.columns = columns
df_test.columns = columns

Теперь наши данные выглядят красивее:

In [5]:
print(df_train.head(5))

   label  qid  feature_1  feature_2  feature_3  feature_4  feature_5  \
0    2.0    1          3          3          0          0          3   
1    2.0    1          3          0          3          0          3   
2    0.0    1          3          0          2          0          3   
3    2.0    1          3          0          3          0          3   
4    1.0    1          3          0          3          0          3   

   feature_6  feature_7  feature_8  ...  feature_127  feature_128  \
0        1.0        1.0   0.000000  ...           62     11089534   
1        1.0        0.0   1.000000  ...           54     11089534   
2        1.0        0.0   0.666667  ...           45            3   
3        1.0        0.0   1.000000  ...           56     11089534   
4        1.0        0.0   1.000000  ...           64            5   

   feature_129  feature_130  feature_131  feature_132  feature_133  \
0            2          116        64034           13            3   
1           

Проведем небольшой EDA.

Всего у нас 10000 документов в трейне:

In [6]:
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 138 entries, label to feature_136
dtypes: float64(97), int64(41)
memory usage: 10.5 MB
None


И 10000 документов в тесте:

In [7]:
print(df_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 138 entries, label to feature_136
dtypes: float64(97), int64(41)
memory usage: 10.5 MB
None


Сколько у нас запросов?

In [8]:
num_queries_train = df_train['qid'].nunique()
num_queries_test = df_test['qid'].nunique()
print(f"Got {num_queries_train} train and {num_queries_test} test queries")

Got 87 train and 88 test queries


Получается, у нас примерно по 100 документов на запрос.

Это типично, когда, например, для сбора датасета обкачивались и заливались на оценку топ-100 документов поисковой выдачи по случайным запросам.

Теперь посмотрим на распределение таргетов (оценок):

In [9]:
print(df_train['label'].value_counts())

label
0.0    5481
1.0    3000
2.0    1326
3.0     142
4.0      51
Name: count, dtype: int64


In [10]:
print(df_test['label'].value_counts())

label
0.0    5755
1.0    2830
2.0    1221
3.0     148
4.0      46
Name: count, dtype: int64


Теперь нам надо представить датасет в формате, который можно подавать на вход катбустовой модели.  
Для этого придется разделить его на 3 части:

- **y** -- вектор таргетов
- **X** -- тензор из фичей
- **q** -- вектор из ID запросов, которые позволяют сгруппировать все документы, которые относятся к одному и тому же запросу

CatBoost требует, чтобы в векторе **q** одинаковые ID запроса шли подряд (но в отличие от, например, xgboost, не требует их строгой сортированности). Однако в нашем случае никаких дополнительных действий не потребуется т.к. датасет уже и так отсортирован по qid.

Убедимся в этом:

In [11]:
print(df_train['qid'].is_monotonic_increasing)
print(df_test['qid'].is_monotonic_increasing)

True
True


Сконвертируем датасет в нужный формат:

In [12]:
def to_catboost_dataset(df):
    y = df['label'].to_numpy()                       # Label: [0-4]
    q = df['qid'].to_numpy().astype('uint32')        # Query Id
    X = df.drop(columns=['label', 'qid']).to_numpy() # 136 features
    return (X, y, q)

X_train, y_train, q_train = to_catboost_dataset(df_train)
X_test, y_test, q_test = to_catboost_dataset(df_test)

## Обучаем pointwise модель

Теперь можно приступить непосредственно к обучению модели. Мы начнем с простой pointwise модели которая в качестве лосса использует обычное RMSE.

Подготовим пулы катбуста:

In [13]:
pool_train = catboost.Pool(data=X_train, label=y_train, group_id=q_train)
pool_test = catboost.Pool(data=X_test, label=y_test, group_id=q_test)

Зададим целевую метрику, которую будем оптимизировать.  
В нашем случае будем использовать NDCG@10:

In [14]:
EVAL_METRIC = 'NDCG:top=10;type=Exp'

Подготовим параметры обучения модели, в т.ч.:
- целевую метрику
- сид генератора случайных чисел
- число итераций, после которого останавливаем обучение если в течение данного числа итераций мы не наблюдаем улучшения целевой метрики на валидационном множестве

Если хотим обучаться на GPU, то еще надо добавить параметр *task_type=GPU*

In [15]:
DEFAULT_PARAMS = {
    'iterations': 1000,            # maximum possible number of trees
    'early_stopping_rounds': 100,  # stop if metric does not improve for N rounds
    'eval_metric': EVAL_METRIC,    # # metric used for early stopping
    'random_seed': 22,
    'verbose': 10
}

Мы будем обучать разные модели, использующие разные лоссы, соответствующие разным алгоритмам машинного обучения ранжированию.  
Напишем функцию, которая позволит кастомизировать модель под нужный лосс:

In [16]:
def create_model(loss_function):
    params = copy.deepcopy(DEFAULT_PARAMS)

    # Temporary directory that is used by catboost to store additional information
    catboost_info_dir = f"/tmp/catboost_info.{loss_function.lower()}"

    params.update({
        'loss_function': loss_function,
        'train_dir': str(catboost_info_dir),
    })
    return catboost.CatBoost(params)

Создадим модель:

In [17]:
model = create_model('RMSE')
print(model)

И зафитим ее на нашем обучающем множестве.

Количество деревьев будет выбрано автоматически с использованием т.н. early stopping -- процесс обучения будем остановлен после того, как на валидационном множестве перестанет расти наша целевая метрика (т.е. NDCG). Обратите внимание что в данном случае целевая метрика (NDCG) отличается от лосса, который мы оптизириуем (RMSE).

Для этого передадим в функцию fit() в качестве валидационного множества (параметр eval_set) наш тест-сет.

ВНИМАНИЕ: строго говоря, так делать нельзя т.к. приведет к переобучению. По хорошему, мы должны были сначала разбить наш трейн на собственно обучающее и валидационное множества, и передавать в eval_set уже это валидационное множества. А тест-сет надо было сохранить и использовать уже только в самом конце для подсчета финальных скоров. Однако, для простоты, мы так делать не будем, и оставим все это в качестве упражнения.

In [18]:
# Fit
start = timer()
model.fit(pool_train, eval_set=pool_test, use_best_model=True)
elapsed = timer() - start
print(f"Model fit: num_trees = {model.tree_count_} elapsed = {elapsed:.3f}")

Learning rate set to 0.073096
0:	test: 0.2493536	best: 0.2493536 (0)	total: 85.7ms	remaining: 1m 25s
10:	test: 0.3825128	best: 0.3886914 (9)	total: 166ms	remaining: 14.9s
20:	test: 0.3867278	best: 0.3886914 (9)	total: 245ms	remaining: 11.4s
30:	test: 0.3896056	best: 0.3896056 (30)	total: 337ms	remaining: 10.5s
40:	test: 0.3973093	best: 0.3983815 (39)	total: 423ms	remaining: 9.9s
50:	test: 0.4030989	best: 0.4063283 (46)	total: 513ms	remaining: 9.55s
60:	test: 0.4085673	best: 0.4085673 (60)	total: 611ms	remaining: 9.41s
70:	test: 0.4053967	best: 0.4108300 (65)	total: 694ms	remaining: 9.08s
80:	test: 0.4091555	best: 0.4134532 (75)	total: 786ms	remaining: 8.92s
90:	test: 0.4050574	best: 0.4134532 (75)	total: 854ms	remaining: 8.53s
100:	test: 0.4087148	best: 0.4134532 (75)	total: 922ms	remaining: 8.21s
110:	test: 0.4114762	best: 0.4134532 (75)	total: 994ms	remaining: 7.96s
120:	test: 0.4098458	best: 0.4140502 (112)	total: 1.08s	remaining: 7.88s
130:	test: 0.4117132	best: 0.4140502 (112)	tot

Видим, что модель состоит из 308 деревьев, и лучший скор NDCG@10 на тесте равен **0.419**

При желании, мы теперь можем сохранить модель в формате cbm:

In [19]:
model_file = "/tmp/model.cbm"

# Save model
model.save_model(model_file)

# Load model
# model = catboost.CatBoost()
# model.load_model(model_file)

Получим предикты модели на тестовом множестве:

In [20]:
y_hat_test = model.predict(pool_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

Predicted: y_hat_test.shape = (10000,)


Теперь, имея предикты, можно посчитать метрики качества:

In [21]:
def compute_metrics(y_true, y_hat, q):
    # List of metrics to evaluate
    eval_metrics = ['NDCG:top=10;type=Exp', 'DCG:top=10;type=Exp', 'MAP:top=10']
    
    for eval_metric in eval_metrics:
        scores = utils.eval_metric(y_true, y_hat, eval_metric, group_id=q)
    
        # Print scores
        print(f"metric = {eval_metric} score = {scores[0]:.3f}")
    
# Compute metrics on test
compute_metrics(y_test, y_hat_test, q_test)

metric = NDCG:top=10;type=Exp score = 0.419
metric = DCG:top=10;type=Exp score = 8.600
metric = MAP:top=10 score = 0.524


Мы видим, что значение NDCG@10 на тесте совпало с тем, что вывел сам катбуст во время обучения модели!

## Обучаем YetiRank

Теперь проделаем все то же самое, но на этот раз с использованием алгоритма YetiRank:

In [22]:
# Create model
model = create_model('YetiRank')

# Fit
start = timer()
model.fit(pool_train, eval_set=pool_test, use_best_model=True)
elapsed = timer() - start
print(f"Model fit: elapsed = {elapsed:.3f} num_trees = {model.tree_count_}")

# Predict
y_hat_test = model.predict(pool_test)
print(f"Predicted: y_hat_test.shape = {y_hat_test.shape}")

# Compute metrics on test
print("\nEvaluated:")
compute_metrics(y_test, y_hat_test, q_test)

0:	test: 0.2873437	best: 0.2873437 (0)	total: 50ms	remaining: 49.9s
10:	test: 0.3811494	best: 0.3828752 (9)	total: 244ms	remaining: 22s
20:	test: 0.3886236	best: 0.3914324 (15)	total: 410ms	remaining: 19.1s
30:	test: 0.3891378	best: 0.3914324 (15)	total: 563ms	remaining: 17.6s
40:	test: 0.3946586	best: 0.3954174 (34)	total: 722ms	remaining: 16.9s
50:	test: 0.4024181	best: 0.4024181 (50)	total: 882ms	remaining: 16.4s
60:	test: 0.4049030	best: 0.4049030 (60)	total: 1.05s	remaining: 16.1s
70:	test: 0.4146104	best: 0.4151574 (69)	total: 1.21s	remaining: 15.8s
80:	test: 0.4153313	best: 0.4167823 (73)	total: 1.37s	remaining: 15.5s
90:	test: 0.4221178	best: 0.4221178 (90)	total: 1.52s	remaining: 15.2s
100:	test: 0.4244372	best: 0.4253389 (96)	total: 1.68s	remaining: 15s
110:	test: 0.4244111	best: 0.4262510 (102)	total: 1.84s	remaining: 14.7s
120:	test: 0.4231627	best: 0.4273284 (117)	total: 2s	remaining: 14.6s
130:	test: 0.4235735	best: 0.4273284 (117)	total: 2.17s	remaining: 14.4s
140:	test:

Видно, что теперь модель обучается значительно дольше.

Сравним результаты:

- RMSE модель выбила NDCG@10 = **0.419**
- а YetiRank выбивает уже NDCG@10 = **0.439**!

Таким образом мы наглядно видим преимущество pairwise/listwise-подхода над "наивным" pointwise-подходом.